# Predicting fluxes on grid


In [ ]:
import sys
import xarray as xr
import numpy as np
import pandas as pd
from joblib import load
from matplotlib import pyplot as plt
from odc.geo.geobox import zoom_out
from odc.algo import xr_reproject
from datacube.utils.dask import start_local_dask

sys.path.append('/g/data/os22/chad_tmp/climate-carbon-interactions/src/')
from _collect_prediction_data import collect_prediction_data, round_coords

sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.classification import predict_xr, HiddenPrints

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

## Analysis Parameters

In [ ]:
var = 'NEE'
suffix='20230817'

t1, t2='1982','2022'
rescale=False

### Set up paths

In [ ]:
results_name=var+'_'+t1+'_'+t2+'_5km_LGBM_'+suffix+'.nc'
model_path = '/g/data/os22/chad_tmp/climate-carbon-interactions/results/models/fluxes/'+var+'_LGBM_model_'+suffix+'.joblib'
# mask_path = '/g/data/os22/chad_tmp/NEE_modelling/data/1km/mask_1km_monthly_2003_2021.nc'
features_list = '/g/data/os22/chad_tmp/climate-carbon-interactions/results/variables_'+suffix+'.txt'

## Open model

In [ ]:
model = load(model_path).set_params(n_jobs=1)

## Open predictor data

In [ ]:
## open data
data = collect_prediction_data(time_start=t1,
                             time_end=t2,
                             verbose=False,
                             export=False,
                             chunks=dict(latitude=1000, longitude=1000, time=1)
                             )

## Create a no-data mask

If we haven't alreday

In [ ]:
# mask = data[['NDVI', 'WCF', 'PV', 'tavg']].to_array().isnull().any('variable')
# mask.compute().to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/mask_5km_monthly_1982_2022.nc')
mask = xr.open_dataarray('/g/data/os22/chad_tmp/climate-carbon-interactions/data/mask_5km_monthly_1982_2022.nc')

In [ ]:
# #create an urban mask once, then next time load it.
# mask1 = xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/data/urban_mask_1km.nc')
# mask1 = mask1.odc.reproject(mask.odc.geobox, resampling='mode')
# mask1=round_coords(mask1)
# mask1.name='urban_mask'
# mask1.compute().to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/urban_mask_5km_.nc')

### Index by variables and check variable order

In [ ]:
train_vars = list(pd.read_csv(features_list))[0:-1]
train_vars=[i[:-3] for i in train_vars]

data = data[train_vars]

if train_vars == list(data.data_vars):
    print('Variables match, n:', len(data.data_vars))
else:
    raise ValueError("Variables don't match")

### Predict each time-step seperately

- TO DO: fix timesteps that come back from `predict_xr`

In [ ]:
# 1157 - 1202

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")

results = []
i=0

for i in range(0, len(data.time))[0:12]: 
    print(" {:03}/{:03}\r".format(i + 1, len(range(0, len(data.time)))), end="")
    with HiddenPrints():
        predicted = predict_xr(model,
                            data.isel(time=i),
                            proba=False,
                            clean=True,
                            # chunk_size=50000,
                              ).compute()
    
    predicted = predicted.Predictions.where(~mask.isel(time=i))
    predicted['time'] = data.isel(time=i).time.values
    results.append(predicted.astype('float32'))
    i+=1 

In [ ]:
#time to run full prediction if lazy input data
print((75/12)*492/60, 'minutes')

#time to run full prediction if NOT lazy input data
print((69/12)*492/60, 'minutes')

In [ ]:
ds = xr.concat(results, dim='time').sortby('time').rename(var).astype('float32')

## Mask urban areas using landcover dataset

In [ ]:
mask1 = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/urban_mask_5km.nc')['urban_mask']
mask1 = mask1.rename({'latitude':'y', 'longitude':'x'})
ds = ds.where(mask1!=1).astype('float32')

In [ ]:
# ds.plot.imshow(col='time', col_wrap=3, robust=True);

### Save results

In [ ]:
ds.to_netcdf('/g/data/os22/chad_tmp/NEE_modelling/results/predictions/'+results_name)

## Animate results

In [ ]:
import xarray as xr
from IPython.display import Image
import matplotlib.pyplot as plt

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools')
from dea_tools.plotting import xr_animation


In [ ]:
# ds = xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/results/predictions/GPP_2003_2021_5km_LGBM.nc')

In [ ]:
path = '/g/data/os22/chad_tmp/NEE_modelling/results/figs/'+var+'_mystudy_LGBM_5km_'+suffix+'.gif'

if var=='NEE':
    imshow={'vmin': -50, 'vmax': 50, 'cmap': 'RdBu_r'}
    
else:
    imshow={'vmin': 0, 'vmax': 150, 'cmap': 'viridis'}

xr_animation(ds.to_dataset(),
            bands=[var],
            show_date='%b %Y',
            width_pixels=600,
            output_path=path,
            show_colorbar=True,
            colorbar_kwargs={'colors': 'black'},
            # show_gdf=poly_gdf,
            interval=200, 
            show_text=var+' gC/m2/month',
            # gdf_kwargs={'edgecolor': 'grey', 'linewidth':0.5}, 
            imshow_kwargs=imshow
            )

# Plot animation
plt.close()
Image(path, embed=True)